<a href="https://colab.research.google.com/github/ANIZAI/Machine-Learning-based-Anomaly-Detection/blob/main/Copy_of_new_dataset__lstm_anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#imports_part_1.py
from pathlib import Path # convenient way to deal w/ paths
import plotly.graph_objects as go # creates plots
import numpy as np # standard for data processing
import pandas as pd # standard for data processing
import json # we have anomalies' timestamps in json format

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
labels_filepath = '/content/drive/MyDrive/final/combined_labels.json'
 
training_filename = '/content/drive/MyDrive/final/train.csv'
 
valid_filename = '/content/drive/MyDrive/final/valid.csv'

In [ ]:
#labels_loading.py
with open(labels_filepath, 'r') as f:
    anomalies_timestamps = json.load(f)

In [ ]:
#read_data.py 
train = pd.read_csv(training_filename)
valid = pd.read_csv(valid_filename)

In [ ]:
train.head()

,timestamp,value
0,2014-03-07 03:41:00,45.868
1,2014-03-07 03:46:00,47.606
2,2014-03-07 03:51:00,42.580
3,2014-03-07 03:56:00,46.030
4,2014-03-07 04:01:00,44.992


In [ ]:
valid.head()

,timestamp,value
0,2014-02-14 14:27:00,51.846
1,2014-02-14 14:32:00,44.508
2,2014-02-14 14:37:00,41.244
3,2014-02-14 14:42:00,48.568
4,2014-02-14 14:47:00,46.714


In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
def parse_and_standardize(df: pd.DataFrame, scaler: StandardScaler = None):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['stand_value'] = df['value']
    if not scaler:
        scaler = StandardScaler()
        scaler.fit(df['stand_value'].values.reshape(-1, 1))
    df['stand_value'] = scaler.transform(df['stand_value'].values.reshape(-1, 1))
    return scaler

data_scaler = parse_and_standardize(train)
parse_and_standardize(valid, data_scaler)

StandardScaler(copy=True, with_mean=True, with_std=True)

## Get anomalies from the data

In [ ]:
train_anomalies = train[train['timestamp'].isin(anomalies_timestamps[training_filename])]
valid_anomalies = valid[valid['timestamp'].isin(anomalies_timestamps[valid_filename])]

In [ ]:
train_anomalies

,timestamp,value,stand_value
2081,2014-03-14 09:06:00,30.482,-6.416755
3395,2014-03-18 22:41:00,99.248,23.654010
4023,2014-03-21 03:01:00,25.422,-8.629449


In [ ]:
valid_anomalies

,timestamp,value,stand_value
1271,2014-02-19 00:22:00,62.056,7.390276
2930,2014-02-24 18:37:00,34.766,-4.543400


## Plot data with anomalies

### Training data

In [ ]:
import plotly.graph_objects as go

In [ ]:
layout = dict(xaxis=dict(title='Timestamp'), yaxis=dict(title='Battery Temperature'))
fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter(x=train['timestamp'], y=train['value'], 
                         mode='markers', name='Non-anomaly',
                         marker=dict(color='blue')))
fig.add_trace(go.Scatter(x=train_anomalies['timestamp'], y=train_anomalies['value'], 
                         mode='markers', name='Anomaly',
                         marker=dict(color='green', size=13)))

## Label anomalies and non-anomalies 

In [ ]:
train['anomaly'] = 0
train.loc[train_anomalies.index, 'anomaly'] = 1
train.iloc[train_anomalies.index]

,timestamp,value,stand_value,anomaly
2081,2014-03-14 09:06:00,30.482,-6.416755,1
3395,2014-03-18 22:41:00,99.248,23.654010,1
4023,2014-03-21 03:01:00,25.422,-8.629449,1


In [ ]:
valid['anomaly'] = 0
valid.loc[valid_anomalies.index, 'anomaly'] = 1
valid.iloc[valid_anomalies.index]

,timestamp,value,stand_value,anomaly
1271,2014-02-19 00:22:00,62.056,7.390276,1
2930,2014-02-24 18:37:00,34.766,-4.543400,1


In [ ]:
train.head()

,timestamp,value,stand_value,anomaly
0,2014-03-07 03:41:00,45.868,0.311407,0
1,2014-03-07 03:46:00,47.606,1.071419,0
2,2014-03-07 03:51:00,42.580,-1.126407,0
3,2014-03-07 03:56:00,46.030,0.382248,0
4,2014-03-07 04:01:00,44.992,-0.071660,0


# Dataset Preparation for LSTM

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [ ]:
class CPUDataset(Dataset):
    def __init__(self, data: pd.DataFrame, size: int):
        self.chunks = torch.FloatTensor(data['stand_value']).unfold(0, size, size)
        
    def __len__(self):
        return self.chunks.size(0)
    
    def __getitem__(self, i):
        x = self.chunks[i]
        return x

In [ ]:
train_values = train['stand_value'].values.astype(np.float32).flatten()
valid_values = valid['stand_value'].values.astype(np.float32).flatten()

In [ ]:
train_ds = CPUDataset(train, 64)
valid_ds = CPUDataset(valid, 64)

train_dataloader = DataLoader(train_ds, batch_size=1)
validation_dataloader = DataLoader(valid_ds, batch_size=1)

# LSTM Architecture

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, in_size, hidden_size, out_size, device):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(in_size, hidden_size)
        self.linear = nn.Linear(hidden_size, out_size)
        self.device = device
        self.init_hidden()
        
    def forward(self, x):
        out, self.hidden_state = self.lstm(x.view(len(x), 1, -1), self.hidden_state)
        self.hidden_state = tuple([h.detach() for h in self.hidden_state])
        out = out.view(len(x), -1)
        out = self.linear(out)
        return out
    
    def init_hidden(self):
        self.hidden_state = (torch.zeros((1, 1, self.hidden_size)).to(self.device),
                             torch.zeros((1, 1, self.hidden_size)).to(self.device))

# Training of the model

In [ ]:
from tqdm.notebook import tqdm
import torch.optim as opt
import math
import copy

Definition of the training loop

In [ ]:
def train_model(model: LSTMModel, dataloaders: dict, optimizer: opt.Optimizer, 
                scheduler, criterion, device: torch.device, epochs: int):
    losses_data = {'train': [], 'valid': []}
    model.to(device)
    for epoch in tqdm(range(epochs)):
        print(f'Epoch {epoch}/{epochs-1}')
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.
            running_total = 0.
            
            for idx, sequence in enumerate(dataloaders[phase]):
                value = sequence
                value = value.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    out = model(value.view(-1, 1))
                    loss = criterion(out.view(-1), value.view(-1))

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        scheduler.step()

                running_loss += loss.item() * out.size(0)
                running_total += out.size(0)

            epoch_loss = running_loss / running_total
            print(f'{phase.capitalize()} Loss: {epoch_loss}')
            losses_data[phase].append(epoch_loss)
    return losses_data

Initialization of the model, dataloaders and training parameters

In [ ]:
total_epoch_count = 50

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(1, 128, 1, device)
model = model.to(device)

dataloaders = {
    'train': train_dataloader,
    'valid': validation_dataloader
}

optimizer = opt.Adam(params=model.parameters(), lr=1e-3)
sched = opt.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-3, steps_per_epoch=len(dataloaders['train']), epochs=total_epoch_count)
criterion = nn.MSELoss()

Training

In [ ]:
losses_dict = train_model(
    model,
    dataloaders,
    optimizer,
    sched,
    criterion,
    device,
    total_epoch_count
)

Epoch 0/49
Train Loss: 1.0092880848854306
Valid Loss: 4.402401396206447
Epoch 1/49
Train Loss: 0.9949247614731864
Valid Loss: 4.259160143988473
Epoch 2/49
Train Loss: 0.9755909608470069
Valid Loss: 4.020755410194397
Epoch 3/49
Train Loss: 0.9438784519831339
Valid Loss: 3.5299635576823403
Epoch 4/49
Train Loss: 0.8778007508270325
Valid Loss: 2.793155210358756
Epoch 5/49
Train Loss: 0.7892414832872058
Valid Loss: 1.8299469569372753
Epoch 6/49
Train Loss: 0.5149271009223801
Valid Loss: 1.3572124564458454
Epoch 7/49
Train Loss: 0.2029424026490204
Valid Loss: 0.3564397167000506
Epoch 8/49
Train Loss: 0.10523011679446975
Valid Loss: 0.11590001032879901
Epoch 9/49
Train Loss: 0.08631100177548692
Valid Loss: 0.05520507093105051
Epoch 10/49
Train Loss: 0.07409838733835226
Valid Loss: 0.03727697567009027
Epoch 11/49
Train Loss: 0.06236257500154713
Valid Loss: 0.020154227413946672
Epoch 12/49
Train Loss: 0.05324084501776342
Valid Loss: 0.012958837066969228
Epoch 13/49
Train Loss: 0.04528639703598

Plot of the training and validation losses

In [ ]:
layout = dict(xaxis=dict(title='Epoch'), yaxis=dict(title='Loss'))
fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter(y=losses_dict['train'], mode='lines', name='Train Loss',))
fig.add_trace(go.Scatter(y=losses_dict['valid'], mode='lines', name='Valid Loss'))

In [ ]:
print(min(*losses_dict['train']))
print(min(*losses_dict['valid']))

0.004068385847053904
0.0012459561878044735


# Results Evaluation

Getting of the 'pure' result

In [ ]:
layout = dict(xaxis=dict(title='Timestamp'), yaxis=dict(title='Battery Temperature'))
fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter(x=train['timestamp'], y=train['stand_value'], 
                         mode='markers', name='Ground Truth',
                         marker=dict(color='blue')))
fig.add_trace(go.Scatter(x=train['timestamp'], y=torch.flatten(res_train), 
                         mode='markers', name='Predicted Value', 
                         marker=dict(color='orange')))

## Anomaly detection with one threshold

We use **three-sigma rule** applied to model's prediction errors to detect anomalies

### Threshold calculation

Calculation of the prediction errors for **training** data *(and only for training)*

In [ ]:
def calculate_prediction_errors(target, predicted, criterion):
    reconstruction_errors = []
    for t, p in zip(target, predicted):
        reconstruction_errors = np.append(
            reconstruction_errors, criterion(p, t).cpu().numpy().flatten()
        )

    return reconstruction_errors


In [ ]:
train_pred_errors = calculate_prediction_errors(res_train.view(-1), torch.tensor(train_values).view(-1), criterion)
train_pred_errors.shape

(4032,)

In [ ]:
np.mean(train_pred_errors), np.std(train_pred_errors)

(0.004068322484567026, 0.25515292776799664)

The threshold is calculated as the mean of the prediction errors + 3 standard deviations of them

In [ ]:
pred_error_threshold = np.mean(train_pred_errors) + 3 * np.std(train_pred_errors)
pred_error_threshold

0.7695271057885569

In [ ]:
def detect_anomalies(predicted: torch.Tensor, target, threshold: float, criterion):
    anomalies_idxs = []
    for idx, (t, p) in enumerate(zip(target, predicted)):
        error = criterion(p.view(-1), t.view(-1)).cpu().numpy().flatten()
        if error > threshold:
            anomalies_idxs.append(idx)
    return anomalies_idxs

In [ ]:
train_anomalies_idxs = detect_anomalies(res_train, torch.tensor(train_values), pred_error_threshold, criterion)
train_anomalies_idxs

[3395]

Plot of the result for training data:


*   Blue points - non-anomaly data
*   Red points - detected anomaly data
*   Green points - real anomaly data

In [ ]:
layout = dict(xaxis=dict(title='Timestamp'), yaxis=dict(title='Battery Temperature'))
fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter(x=train['timestamp'], y=train['value'], 
                         mode='markers', name='Non-anomaly',
                         marker=dict(color='blue')))
fig.add_trace(go.Scatter(x=train_anomalies['timestamp'], y=train_anomalies['value'], 
                         mode='markers', name='Real Anomaly',
                         marker=dict(color='green', size=13)))
fig.add_trace(go.Scatter(x=train['timestamp'][train_anomalies_idxs],
                         y=train['value'][train_anomalies_idxs], 
                         mode='markers', name='Detected Anomaly',
                         marker=dict(color='red', size=7)))

In [ ]:
valid_anomalies_idxs = detect_anomalies(res_valid, torch.tensor(valid_values), pred_error_threshold, criterion)
valid_anomalies_idxs

[1271]

### Metrics calculation

Finally, we calculate several metrics for the model with one threshold:


*   Confusion matrix
*   Precision
*   Recall
*   F-beta score

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
def calculate_metrics(ground_truth: pd.DataFrame, anomalies_idxs: list):
    predictions = pd.DataFrame(index=range(len(ground_truth)), columns=['predicted_anomaly'])
    predictions['predicted_anomaly'] = 0
    predictions.iloc[anomalies_idxs] = 1
    
    confusion_matrix = pd.crosstab(ground_truth.loc[:, 'anomaly'], predictions['predicted_anomaly'], margins=True)
    precision, recall, f1, _ = precision_recall_fscore_support(
        ground_truth.loc[:, 'anomaly'], predictions['predicted_anomaly'], beta=2., average='binary'
    )
    return confusion_matrix, precision, recall, f1

In [ ]:
print(f'Train:\n Precision: {train_metrics[0]:.3f}\n Recall: {train_metrics[1]:.3f}\n F1 score: {train_metrics[2]:.3f}')

Train:
 Precision: 1.000
 Recall: 0.333
 F1 score: 0.385


## Anomaly detection with dynamic threshold

Dynamic threshold is calculated for each point depending on mean and standart deviation in window around this point

### Threshold calculation

Definition of the window and coefficient for standard deviation, based on which the threshold is calculated

In [ ]:
window = 40
std_coef = 6

Calculation of the dynamic threshold using the prediction errors for **training** data

In [ ]:
train_pred_errors_windowed = pd.Series(train_pred_errors).rolling(window=window, min_periods=1)
train_dynamic_threshold = train_pred_errors_windowed.mean() + std_coef * train_pred_errors_windowed.std()

Calculation of the dynamic threshold using the prediction errors for **validation** data

In [ ]:
valid_pred_errors = calculate_prediction_errors(res_valid.view(-1), torch.tensor(valid_values).view(-1), criterion)

valid_pred_errors_windowed = pd.Series(valid_pred_errors).rolling(window=window, min_periods=1)
valid_dynamic_threshold = valid_pred_errors_windowed.mean() + std_coef * valid_pred_errors_windowed.std()

### Data filtering

Then, we filter results of the model according to the thresholds and get the **indexes** of detected anomalies

In [ ]:
def detect_anomalies(predicted: torch.Tensor, target, upper_bound, criterion):
    anomalies_idxs = []
    for idx, (t, p, u) in enumerate(zip(target, predicted, upper_bound)):
        error = criterion(p.view(-1), t.view(-1)).cpu().numpy().flatten()
        if error > u:
            anomalies_idxs.append(idx)
    return anomalies_idxs

In [ ]:
train_anomalies_dynamic_idxs = detect_anomalies(res_train, torch.tensor(train_values), train_dynamic_threshold, criterion)
train_anomalies_dynamic_idxs

[1296, 2011, 2081, 2777, 3391, 3394, 3395, 3980, 4023]

In [ ]:
valid_anomalies_dynamic_idxs = detect_anomalies(res_valid, torch.tensor(valid_values), valid_dynamic_threshold, criterion)
valid_anomalies_dynamic_idxs

[1271, 2929, 2931, 2971]

Plot of the result for training data:


*   Blue points - non-anomaly data
*   Red points - detected anomaly data
*   Green points - real anomaly data

In [ ]:
layout = dict(xaxis=dict(title='Timestamp'), yaxis=dict(title='Battery Temperature'))
fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter(x=train['timestamp'], y=train['value'], 
                         mode='markers', name='Non-anomaly',
                         marker=dict(color='blue')))
fig.add_trace(go.Scatter(x=train_anomalies['timestamp'], y=train_anomalies['value'], 
                         mode='markers', name='Real Anomaly',
                         marker=dict(color='green', size=13)))
fig.add_trace(go.Scatter(x=train['timestamp'][train_anomalies_dynamic_idxs],
                         y=train['value'][train_anomalies_dynamic_idxs], 
                         mode='markers', name='Detected Anomaly',
                         marker=dict(color='red', size=7)))

### Metrics calculation

Finally, we calculate several metrics for the model with dynamic threshold:


*   Confusion matrix
*   Precision
*   Recall
*   F-beta score

Metrics for training data

In [ ]:
print(f'Train:\n Precision: {train_metrics[0]:.3f}\n Recall: {train_metrics[1]:.3f}\n F1 score: {train_metrics[2]:.3f}')

Train:
 Precision: 0.333
 Recall: 1.000
 F1 score: 0.714
